There are a few potential stumbling blocks before beginning. PyTorch MPS requires MacOS 12.3 or later and an ARM Python installation.
This can be checked by using the platform module:

In [1]:
import platform
platform.platform()

'macOS-12.3-arm64-arm-64bit'

Next it can be installed:
```python
# MPS acceleration is available on MacOS 12.3+
pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu
```
- torch: https://download.pytorch.org/whl/nightly/cpu/torch-1.13.0.dev20220701-cp39-none-macosx_11_0_arm64.whl

- torchvision: https://download.pytorch.org/whl/nightly/cpu/torchvision-0.14.0.dev20220701-cp39-cp39-macosx_11_0_arm64.whl

- torchaudio: https://download.pytorch.org/whl/nightly/cpu/torchaudio-0.14.0.dev20220603-cp39-cp39-macosx_11_0_arm64.whl

Next we confirm that our torch installation has access to MPS/Metal:

In [2]:
import torch

torch.has_mps

True

In [5]:
!pip install transformers datasets

  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
  Using cached datasets-2.3.2-py3-none-any.whl (362 kB)
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached regex-2022.6.2-cp39-cp39-macosx_11_0_arm64.whl (281 kB)
  Using cached fsspec-2022.5.0-py3-none-any.whl (140 kB)
  Using cached xxhash-3.0.0-cp39-cp39-macosx_11_0_arm64.whl (30 kB)
  Using cached multiprocess-0.70.13-py39-none-any.whl (132 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached pandas-1.4.3-cp39-cp39-macosx_11_0_arm64.whl (10.5 MB)
  Using cached pyarrow-8.0.0-cp39-cp39-macosx_11_0_arm64.whl (16.2 MB)
  Using cached aiohttp-3.8.1-cp39-cp39-macosx_11_0_arm64.whl (552 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 

In [3]:
from datasets import load_dataset  # pip install datasets

# load the first 1K rows of the TREC dataset
trec = load_dataset('trec', split='train[:1000]')
trec

ModuleNotFoundError: No module named 'datasets'

In [ ]:
from transformers import AutoTokenizer, AutoModel  # pip install transformers

# initialize BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# take the first 64 rows of the trec data
text = trec['text'][:64]
# tokenize text using the BERT tokenizer
tokens = tokenizer(
    text, max_length=512,
    truncation=True, padding=True,
    return_tensors='pt'
)

In [ ]:
device = torch.device('mps')
model.to(device)
tokens.to(device)
device

In [ ]:
%%timeit
model(**tokens)

In [ ]:

from transformers import BertForSequenceClassification, BertConfig

config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = max(trec['label-coarse'])+1  # create 6 outputs
model = BertForSequenceClassification(config).to(device)
# remember to move to MPS with .to(device)

In [ ]:
# activate training mode of model
model.train()

# initialize adam optimizer
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

# begin training loop
for batch in loader:
  	# note that we move everything to the MPS device
    batch_mps = {
        'input_ids': batch['input_ids'].to(device),
        'attention_mask': batch['attention_mask'].to(device),
        'labels': batch['labels'].to(device)
    }
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # train model on batch and return outputs (incl. loss)
    outputs = model(**batch_mps)
    # extract loss
    loss = outputs[0]
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()